In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
!ln -s '/content/gdrive/My Drive/' /mydrive
%cd /mydrive

In [ ]:
!git clone https://github.com/terminalai/STEAMxD.git
!pip install -qr STEAMxD/yolov5/requirements.txt

from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate STEAMxD/yolov5/data/baseline.yaml
path: ../datasets/baseline2/
train: train/images
val: valid/images
test:

names:
  0: humans
  1: pets

In [ ]:
%%writetemplate STEAMxD/yolov5/models/baseline.yaml

# Parameters
nc: 2  # number of classes
depth_multiple: 0.67  # model depth multiple
width_multiple: 0.75  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
hsv_h =  0.015  # image HSV-Hue augmentation (fraction)
hsv_s = 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v = 0.4  # image HSV-Value augmentation (fraction)
degrees = 20.0  # image rotation (+/- deg)
translate = 0.1  # image translation (+/- fraction)
scale = 0.6  # image scale (+/- gain)
shear = 0.0  # image shear (+/- deg)
perspective = 0.000  # image perspective (+/- fraction), range 0-0.001
flipud = 0.0  # image flip up-down (probability)
fliplr = 0.0  # image flip left-right (probability)
mosaic = 0.5  # image mosaic (probability)
mixup = 0.0  # image mixup (probability)

In [ ]:
%%writetemplate STEAMxD/yolov5/data/hyps/hyp.scratch-low.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license
# Hyperparameters for low-augmentation COCO training from scratch
# python train.py --batch 64 --cfg yolov5n6.yaml --weights '' --data coco.yaml --img 640 --epochs 300 --linear
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

#################################
### DO NOT ALTER THIS PORTION ###
#################################
lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.01  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
#################################
### DO NOT ALTER THIS PORTION ###
#################################
###########################################
### Image Augmentation Parameters Start ###
###########################################
hsv_h: {hsv_h}  # image HSV-Hue augmentation (fraction)
hsv_s: {hsv_s}  # image HSV-Saturation augmentation (fraction)
hsv_v: {hsv_v}  # image HSV-Value augmentation (fraction)
degrees: {degrees}  # image rotation (+/- deg)
translate: {translate}  # image translation (+/- fraction)
scale: {scale}  # image scale (+/- gain)
shear: {shear} # image shear (+/- deg)
perspective: {perspective}  # image perspective (+/- fraction), range 0-0.001
flipud: {flipud}  # image flip up-down (probability)
fliplr: {fliplr}  # image flip left-right (probability)
mosaic: {mosaic}  # image mosaic (probability)
mixup: {mixup}  # image mixup (probability)
#########################################
### Image Augmentation Parameters End ###
#########################################
# copy_paste is for image segmentation leave this augmentation alone
copy_paste: 0.0  # segment copy-paste (probability)

In [ ]:
!python STEAMxD/yolov5/train.py --img 640 --batch 16 --epochs 200 --data STEAMxD/yolov5/data/baseline.yaml --cfg STEAMxD/yolov5/models/baseline.yaml --weights yolov5s.pt --cache --name baseline1 --project 'STEAMxD/yolov5/runs/train'

train: weights=yolov5s.pt, cfg=yolov5/models/baseline.yaml, data=yolov5/data/baseline.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=sample, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 2023-1-16 Python-3.8.16 torch-1.13.0+cu116 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, a